# create master json files from the raw data

In [ ]:
import json
import numpy as np
import os
import sys
import time
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)
import src.data_pipeline as dp

In [ ]:
# Example usage:
path = 'C:/Users/anedaeij23/Project/tooth_classification_project/data/raw/MOVIPKA-front_right-right-120_PumM5JqRU3.jpg'
dark_result = dp.is_darker_than_threshold(path, 0.25)
print(dark_result)

False


In [81]:
tooth_diag_annots = json.load(open(os.path.join("C:/Users/anedaeij23/Project/tooth_classification_project/data/raw/", "complete_toothwise_annotations.json")))

In [ ]:
new_json = dp.remove_dark_images_from_json(tooth_diag_annots, "C:/Users/anedaeij23/Project/tooth_classification_project/data/raw/" )

In [ ]:
{
    "0": {
        "image_filename": "HONILNO-lower-390.jpg",
        "image_perspective": "lower",
        "image_quality": "G",
        "image_width": 1911,
        "image_height": 1427,
        "teeth_data": {
            "0": {
                "tooth_image_filename": "HONILNO-lower-390_nh1ZWZC_-h.jpg",
                "height": 405,
                "width": 360,
                "bbox": [
                    74,
                    116,
                    344,
                    416
                ],
                "crop_box": [
                    6,
                    86,
                    411,
                    446
                ],
                "tooth_number": 47,
                "region_class": "T47",
                "label_id": "nh1ZWZC_-h",
                "missing": "P",
                "tooth_label": {
                    "all_points_x": [],
                    "all_points_y": [],
                    "shape": "polygon"},
                "diagnostic_labels": {
                    "0": {
                        "region_class": "car",
                        "rle": [...]},
                    ...}},
            ...}},
            ...}

In [97]:
plaque_annotations = dp.convert_annotations(tooth_diag_annots, target_class='Pla')

In [99]:
# Save the new annotations to a file
with open("C:/Users/anedaeij23/Project/tooth_classification_project/data/intrim/plaque_annotations.json", "w") as f:
    json.dump(plaque_annotations, f)